<a href="https://colab.research.google.com/github/Re14m/training/blob/master/2022-0409_recipie157.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [簡単な顔画像認識を機械学習で処理するレシピ](https://axross-recipe.com/recipes/157)

In [ ]:
# パッケージのインストール
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install mglearn
!pip install matplotlib

In [ ]:
# dataset読込(scikit-learn組込の顔画像集)

# min_faces_per_personで一人に対して持ってくる画像の数を20枚に限定,resizeで縮小
from sklearn.datasets import fetch_lfw_people
people = fetch_lfw_people(min_faces_per_person=20,resize=0.7)
image_shape = people.images[0].shape

import matplotlib.pyplot as plt
fig,axes = plt.subplots(2,5,figsize=(15,8),subplot_kw = {"xticks":(),"yticks":()})
for target, image, ax in zip(people.target, people.images, axes.ravel()):
  ax.imshow(image)
  ax.set_title(people.target_names[target])

In [ ]:
# dataのチェック（特徴量から特定人物のデータが多いかどうかを確認する）

# データの内訳
print("総サンプル数,ピクセル:{}".format(people.images.shape))
print("人の数:{}".format(len(people.target_names)))

In [ ]:
# 各サンプルごとの画像数
import numpy as np

counts = np.bincount(people.target)
for i in range(62):
  print(people.target_names[i],counts[i])

In [ ]:
# 特徴量の偏りを制限する（各サンプルごとの画像数を50枚までに制限する）

#ラベルの数だけ０を配置する配列を作成する。
limit = np.zeros(people.target.shape,dtype=np.bool_)

#各ラベルの枚数が上から50枚になるとき、上の配列の0成分を1にする。
for target in np.unique(people.target):
  limit[np.where(people.target == target)[0][:50]]=1

X_people = people.data[limit]
y_people = people.target[limit]

counts = np.bincount(y_people)
for i in range(62):
  print(people.target_names[i],counts[i])

In [ ]:
# datasetの分割（sklearnのtrain_test_splitでデータを訓練セットとテストセットに分割）
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X_people,y_people, stratify=y_people,random_state=42,test_size = 0.2)

In [ ]:
# datasetを主成分分析モデルにかける
from sklearn.decomposition import  PCA

pca = PCA(n_components = 0.9, whiten = True, random_state = 42)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
# モデルにかけた後のdataの可視化（pcaでwhitenパラメータを指定した場合）
import mglearn
mglearn.plots.plot_pca_whitening()

In [ ]:
# データ整形後の特徴量
print(X_train_pca.shape)

In [ ]:
# 整形後のデータを可視化
import matplotlib.pyplot as plt

fig,axes = plt.subplots(3,5,figsize=(15,12),subplot_kw={"xticks":(),"yticks":()})
for i, (component, ax) in enumerate(zip(pca.components_,axes.ravel())):
  ax.imshow(component.reshape(87,65))
  ax.set_title("{}.component".format((i+1)))

In [ ]:
# モデル構築
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(X_train_pca,y_train)

In [ ]:
# 評価の結果

print("訓練セットでの正解率:{:.2f}".format(knn.score(X_train_pca,y_train)))
print("テストセットでの正解率:{:.2f}".format(knn.score(X_test_pca,y_test)))

y_pred = knn.predict(X_test_pca)